In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## このノートブックについて

[YouTube Live Streaming API ](https://developers.google.com/youtube/v3/live/docs/liveChatMessages)でチャットログを収集して任意のスプレッドシートに保存し、センチメント分析を行います。
以下の事前準備で API Key を取得してください。
なお、ログは**配信中のみ**収集可能です。

### 実行手順

1.   [GCP プロジェクト](https://console.cloud.google.com/cloud-resource-manager) を作成します。
2.   [課金設定](https://support.google.com/cloud/answer/6293499#enable-billing) を有効にします。
3.   [YouTube Data API v3 および Cloud Natural Language API](https://console.cloud.google.com/flows/enableapi?apiid=language.googleapis.com,youtube.googleapis.com) を有効にします。
4. [こちらの手順](https://cloud.google.com/docs/authentication/api-keys?hl=ja&visit_id=637671094433160169-4252075124&rd=1#creating_an_api_key)を参考に[API Key](https://console.cloud.google.com/apis/credentials) を作成します。API の制限の欄で、CLoud Natural Language API および YouTube Data API v3が有効になっていることを確認し、クリップボードにキーをコピーしてください。
5. 上記手順で収集した API キーを `API_KEY`、動画のURLを `yt_url`　というパラメータ欄に入力してから、セルの左側にある右向きの三角のアイコンを押して実行します。
6. 画面下部に認証用のURLが表示されますので、リンクを開き、Googleアカウントで認証してから右下の「許可」ボタンを押して、表示されるコードを四角い枠の中にコピー＆ペーストして改行してください。
結果が生成されたらGoogle スプレッドシートの URL が表示されます。

In [4]:
#@title 
#事前に取得したYouTube API key と YouTube のライブ動画 url
API_KEY = '' #@param {type: "string"}
yt_url = "" #@param {type: "string"}
video_id = yt_url.replace('https://www.youtube.com/watch?v=', '')
file=video_id+".csv"

In [ ]:
# 必要なライブラリのインポートと認証
import time
import requests
import json
from datetime import datetime 
from googleapiclient.discovery import build
import pytz

In [16]:
# NLP API でセンチメント分析

def analyze_sentiment(text):
    nl_service = build('language', 'v1beta2', developerKey=API_KEY)
    document_type = "PLAIN_TEXT"
    encoding_type = "UTF8"
    source_sentence = text
    response = nl_service.documents().analyzeSentiment(
     body={
      'document': {
        'type': document_type,
        'content': source_sentence,
        #'language': source_language,
      },
      'encodingType': encoding_type,
     }
    ).execute()
    return response['documentSentiment']['score'], response['documentSentiment']['magnitude']
  # 単語レベルで必要な時
  #for sentence in response['sentences']: 
  #  print(sentence['sentiment'], sentence['text']['content'])

# チャットIDを取得
def get_chat_id(yt_url):
    url    = 'https://www.googleapis.com/youtube/v3/videos'
    params = {'key': API_KEY, 'id': video_id, 'part': 'liveStreamingDetails'}
    data   = requests.get(url, params=params).json()

    liveStreamingDetails = data['items'][0]['liveStreamingDetails']
    if 'activeLiveChatId' in liveStreamingDetails.keys():
        chat_id = liveStreamingDetails['activeLiveChatId']
    else:
        chat_id = None
        print('NOT live')
    return chat_id

# チャットの内容を取得
def get_chat(chat_id, log_file, pageToken):
    url    = 'https://www.googleapis.com/youtube/v3/liveChat/messages'
    params = {'key': API_KEY, 'liveChatId': chat_id, 'part': 'id,snippet,authorDetails'}

    if type(pageToken) == str:
        params['pageToken'] = pageToken
    data   = requests.get(url, params=params).json()
    
    try:
        for item in data['items']:
            dt = datetime.strptime(item['snippet']['publishedAt'], '%Y-%m-%dT%H:%M:%S.%f%z')
            time = dt.astimezone(pytz.timezone("Asia/Tokyo")).strftime('%Y-%m-%d %H:%M:%S') #ISO8601をJSTに
            channelId = item['snippet']['authorChannelId']
            usr       = item['authorDetails']['displayName']
            msg       = item['snippet']['displayMessage']
            sentiment, magnitude = analyze_sentiment(msg)
            """ superchatのデータをとる場合
            if item['snippet']['type'] == "superChatEvent":
                superChat   = item['snippet']['superChatDetails']['amountDisplayString']
            else:
                superChat = ""
            if item['snippet']['type'] == "superStickerEvent":
                superSticker   = item['snippet']['superStickerDetails']['amountDisplayString']
            else:
                superSticker = ""
            """
            #l  = (time, usr, channelId, msg, sentiment, magnitude, superChat, superSticker)
            with open(log_file, 'a') as f:
                 print(time, usr, msg, sentiment, magnitude, sep=',',file=f)
        print('start : ', data['items'][0]['snippet']['publishedAt'])
        print('end   : ', data['items'][-1]['snippet']['publishedAt'])

    except:
        pass
    return data['nextPageToken']

In [18]:
slp_time        = 10 #sec
iter_times      =  90 #90 #回
take_time       = slp_time / 60 * iter_times # 計測時間を分単位に
print('Track {} minutes data'.format(take_time))
print('work on {}'.format(yt_url))

chat_id  = get_chat_id(yt_url)
nextPageToken = None
for i in range(iter_times):
    try:
        nextPageToken = get_chat(chat_id, file, nextPageToken)
        time.sleep(slp_time)
    except:
        break

Track 15.0 minutes data
work on https://www.youtube.com/watch?v=5qap5aO4i9A
start :  2021-12-24T08:58:54.615564+00:00
end   :  2021-12-24T09:01:11.819678+00:00
start :  2021-12-24T09:01:14.054555+00:00
end   :  2021-12-24T09:01:24.92536+00:00
start :  2021-12-24T09:01:29.222264+00:00
end   :  2021-12-24T09:01:38.554852+00:00
start :  2021-12-24T09:01:42.281832+00:00
end   :  2021-12-24T09:01:48.122898+00:00
start :  2021-12-24T09:01:49.238797+00:00
end   :  2021-12-24T09:01:56.464193+00:00
start :  2021-12-24T09:02:04.813358+00:00
end   :  2021-12-24T09:02:10.073017+00:00
start :  2021-12-24T09:02:11.687142+00:00
end   :  2021-12-24T09:02:19.984787+00:00
start :  2021-12-24T09:02:24.112747+00:00
end   :  2021-12-24T09:02:30.882587+00:00
start :  2021-12-24T09:02:37.445812+00:00
end   :  2021-12-24T09:02:41.102274+00:00
start :  2021-12-24T09:02:42.79709+00:00
end   :  2021-12-24T09:02:49.161413+00:00
start :  2021-12-24T09:02:53.466901+00:00
end   :  2021-12-24T09:03:01.466746+00:00
st